In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing

In [2]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

#### Transform environment from `mlagents` to `gymnasium`

In [3]:
import gymnasium as gym
print(gym.__version__)

1.1.1


In [4]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecTransposeImage
from env_camera_gymnasium_wrapper import UnityCameraGymWrapper
from env_camera_raycasts_gymnasium_wrapper import UnityCameraRaycastsGymWrapper

def make_env():

  channel = EngineConfigurationChannel()
  env_path = "D:/_Thesis/warehouse-bot-training/environment_builds/warehouse_stage2_find_potential_dist_spedup/Warehouse_Bot.exe"

  unity_env = UnityEnvironment(
    file_name=env_path,
    side_channels=[channel],
    # additional_args=["-batchmode", "-nographics"]
  )
  
  channel.set_configuration_parameters(time_scale=1)
  
  gymnasium_env = UnityCameraRaycastsGymWrapper(unity_env)
  gymnasium_env = Monitor(gymnasium_env)
  
  print(gymnasium_env.observation_space)
  
  return gymnasium_env

In [5]:
env = DummyVecEnv([make_env])
env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=1.)

Dict('image': Box(0, 255, (3, 36, 64), uint8), 'vector': Box(0.0, 255.0, (100,), float32))


#### Creating stable_baselines3 model

##### Building own network

PPO Hyperparams from mlagents-learn config file

```yaml
behaviors:
  Dlivery_Bot_2:
    trainer_type: ppo
    hyperparameters:
      batch_size: 512
      buffer_size: 2560
      learning_rate: 0.0003
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
      learning_rate_schedule: linear
    network_settings:
      normalize: True
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
    keep_checkpoints: 5
    checkpoint_interval: 100000
    max_steps: 4000000
    time_horizon: 1024
    summary_freq: 10000
    # threaded: False
```

#### Policy Config

In [6]:
import torch as th
import torch.nn as nn
import gymnasium as gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.spaces.Dict, features_dim: int = 256):
        super().__init__(observation_space, features_dim)

        # Shapes of image and vector inputs: [<batch size>, <bands, height, width>], [<batch size>, <length>]
        
        # Create a sub-extractor for each modality
        # For images of size: 3x36x64
        self.image_enc_net = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(2, stride=2),
            nn.Conv2d(16, 16, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(2, stride=2),
            nn.Flatten(),
            nn.Linear(48, 32),
            nn.ReLU(),
        )
        
        # Compute output shape of CNN dynamically
        with th.no_grad():
            sample_image = th.as_tensor(observation_space.spaces["image"].sample()[None]).float()
            cnn_output_size = self.image_enc_net(sample_image).shape[1]
            
        # # Get size of the vector data dynamically
        # with th.no_grad():
        #     vector_input_size = observation_space.spaces["vector"].sample().shape[0]

        # self.vector_enc_net = nn.Sequential(
        #     nn.Linear(vector_input_size, 64),
        #     nn.Linear(64, 12),
        #     nn.ReLU(),
        # )
        
        # # Get  output shape of the vector NN dynamically
        # with th.no_grad():
        #     sample_vector = th.as_tensor(observation_space.spaces["vector"].sample()).float()
        #     vector_network_output_size = self.vector_enc_net(sample_vector).shape[0]

        # # Final linear layer
        # self.linear = nn.Sequential(
        #     nn.Linear(cnn_output_size + vector_network_output_size, features_dim),
        #     nn.ReLU()
        # )

    def forward(self, observations):
        image = observations["image"].float()
        vector = observations["vector"]

        image_features = self.image_enc_net(image)
        return image_features
        # vector_features = self.vector_enc_net(vector)

        # combined = th.cat([image_features, vector_features], dim=1)
        # return self.linear(combined)


#### Decaying Entropy Coefficient

In [7]:
from stable_baselines3.common.callbacks import BaseCallback

class DecayingEntropyCalback(BaseCallback):
    def __init__(self, initial_value=0.1, final_value=0.01, max_steps=100_000, verbose=0):
        super().__init__(verbose)
        self.initial_value = initial_value
        self.final_value = final_value
        self.max_steps = max_steps

    def _on_step(self) -> bool:
        progress = min(1.0, self.num_timesteps / self.max_steps)
        current_ent_coef = self.initial_value * (1.0 - progress) + self.final_value * progress
        self.model.ent_coef = current_ent_coef
        return True

#### Training Algorithm

In [8]:
from stable_baselines3 import PPO

policy_kwargs = dict(
    features_extractor_class=CustomCombinedExtractor,
    features_extractor_kwargs=dict(features_dim=32),
    net_arch=[dict(pi=[20, 12], vf=[16, 8])],
    activation_fn=nn.ReLU
)

model = PPO("MultiInputPolicy",
            env, verbose=1,
            learning_rate=3e-4,
            n_steps=2560,
            batch_size=512,
            n_epochs=10,
            clip_range=0.2,
            gamma=0.99,
            gae_lambda=0.95,
            seed=0,
            ent_coef=0.1,
            vf_coef=0.005,
            policy_kwargs=policy_kwargs,
            # tensorboard_log = './logs/stage2',
            # stats_window_size=1
)

# model = PPO.load('./saved_models/baselines/stage2/find_1_500k.zip', env=gymnasium_env, tensorboard_log='./logs/stage2')

Using cuda device


In [9]:
# model.policy

In [10]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [11]:
print(get_n_params(model.policy))

5960


In [ ]:
model.learn(total_timesteps=500_000, tb_log_name="find_2_potential_dist_4", reset_num_timesteps=False)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 95.7     |
|    ep_rew_mean     | -18.9    |
| time/              |          |
|    fps             | 38       |
|    iterations      | 1        |
|    time_elapsed    | 66       |
|    total_timesteps | 2560     |
---------------------------------


In [ ]:
# decayingEntropyCallback = DecayingEntropyCalback(initial_value=0.02, final_value=0.005, max_steps=100_000)
# model.learn(total_timesteps=100_000, tb_log_name="find_2_potential_dist_4", reset_num_timesteps=False, callback=decayingEntropyCallback)